In [ ]:
import sys
import time
import numpy as np
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import meshcat.geometry as g
import meshcat.transformations as tf

# Add paths
sys.path.append('./python/bsqp')
sys.path.append('./python')

from bsqp.mpc_controller import MPC_GATO
from bsqp.config import PICKPLACE_DEFAULT_GOALS, PENDULUM_DEFAULT_PARAMS, IIWA14_START_CONFIGS, PICKPLACE_SOLVER_PARAMS

np.set_printoptions(linewidth=200)
np.random.seed(42)

In [ ]:
# Robot model
urdf_path = "iiwa_description/iiwa14.urdf"
model_dir = "iiwa_description/"
model, visual_model, collision_model = pin.buildModelsFromUrdf(urdf_path, model_dir)

# MPC parameters
N = 16
dt = 0.01
sim_dt = 0.001

# Goals (can use default or customize)
goals = PICKPLACE_DEFAULT_GOALS
total_time = len(goals) * 5.0

# Starting configuration
x_start = np.hstack((IIWA14_START_CONFIGS['zero'], np.zeros(7)))

# Pendulum configuration
pendulum_config = PENDULUM_DEFAULT_PARAMS.copy()
print(f"Robot: IIWA14 (7-DOF)")
print(f"Goals: {len(goals)}")
print(f"MPC: N={N}, dt={dt}s")
print(f"Pendulum: mass={pendulum_config['mass']}kg, length={pendulum_config['length']}m")

In [ ]:
# Batch size = 1 (no force estimation)
print("\n" + "="*60)
print("Running Batch Size = 1")
print("="*60)

mpc_1 = MPC_GATO(
    model=model,
    model_path=urdf_path,
    N=N,
    dt=dt,
    batch_size=1,
    plant_type='iiwa14',
    pendulum_config=pendulum_config,
    track_full_stats=True,
    solver_params=PICKPLACE_SOLVER_PARAMS
)

_, mpc_stats_1 = mpc_1.run_mpc_goals(
    x_start,
    goals,
    sim_dt=sim_dt,
    goal_timeout=5.0
)

In [ ]:
# Batch size = 8
print("\n" + "="*60)
print("Running Batch Size = 8")
print("="*60)

mpc_8 = MPC_GATO(
    model=model,
    model_path=urdf_path,
    N=N,
    dt=dt,
    batch_size=8,
    plant_type='iiwa14',
    pendulum_config=pendulum_config,
    track_full_stats=True,
    solver_params=PICKPLACE_SOLVER_PARAMS
)

_, mpc_stats_8 = mpc_8.run_mpc_goals(
    x_start,
    goals,
    sim_dt=sim_dt,
    goal_timeout=5.0
)

In [ ]:
# Batch size = 32
print("\n" + "="*60)
print("Running Batch Size = 32")
print("="*60)

mpc_32 = MPC_GATO(
    model=model,
    model_path=urdf_path,
    N=N,
    dt=dt,
    batch_size=32,
    plant_type='iiwa14',
    pendulum_config=pendulum_config,
    track_full_stats=True,
    solver_params=PICKPLACE_SOLVER_PARAMS
)

_, mpc_stats_32 = mpc_32.run_mpc_goals(
    x_start,
    goals,
    sim_dt=sim_dt,
    goal_timeout=5.0
)

In [ ]:
# Batch size = 128
print("\n" + "="*60)
print("Running Batch Size = 128")
print("="*60)

mpc_128 = MPC_GATO(
    model=model,
    model_path=urdf_path,
    N=N,
    dt=dt,
    batch_size=128,
    plant_type='iiwa14',
    pendulum_config=pendulum_config,
    track_full_stats=True,
    solver_params=PICKPLACE_SOLVER_PARAMS
)

_, mpc_stats_128 = mpc_128.run_mpc_goals(
    x_start,
    goals,
    sim_dt=sim_dt,
    goal_timeout=5.0
)

In [ ]:
# Create visualization model with pendulum
model_viz, visual_model_viz, collision_model_viz = pin.buildModelsFromUrdf(urdf_path, model_dir)

# Add pendulum to visualization model
ee_joint_id = model_viz.njoints - 1
pendulum_joint_id = model_viz.addJoint(
    ee_joint_id,
    pin.JointModelSpherical(),
    pin.SE3.Identity(),
    "pendulum_joint"
)

mass = pendulum_config['mass']
length = pendulum_config['length']
com = np.array([0.0, 0.0, -length])
inertia_matrix = np.diag([0.001, 0.001, 0.001])
pendulum_inertia = pin.Inertia(mass, com, inertia_matrix)
model_viz.appendBodyToJoint(pendulum_joint_id, pendulum_inertia, pin.SE3.Identity())

# Initialize visualizer
viz = MeshcatVisualizer(model_viz, collision_model_viz, visual_model_viz)
viz.initViewer(open=True)
viz.loadViewerModel(rootNodeName="robot")

# Add pendulum visualization
viz.viewer['pendulum_rod'].set_object(
    g.Cylinder(height=length, radius=0.01),
    g.MeshLambertMaterial(color=0x808080)
)

viz.viewer['pendulum_bob'].set_object(
    g.Sphere(0.05),
    g.MeshLambertMaterial(color=0x0000ff)
)

print("✓ Meshcat viewer initialized")
print("  Open the meshcat URL above to see visualization")

In [ ]:
# Choose which MPC instance to visualize
# Options: mpc_1, mpc_8, mpc_32, mpc_128
mpc_instance = mpc_128
stats_to_viz = mpc_stats_128

print(f"Visualizing batch_size={mpc_instance.batch_size}")

# Add goal markers with colors based on outcomes
goal_outcomes = stats_to_viz['goal_outcomes']
for i, (goal, outcome) in enumerate(zip(goals, goal_outcomes)):
    color = 0x00ff00 if outcome == 'reached' else 0xff0000
    viz.viewer[f'goal_{i}'].set_object(
        g.Sphere(0.03),
        g.MeshLambertMaterial(color=color)
    )
    T = tf.translation_matrix(goal)
    viz.viewer[f'goal_{i}'].set_transform(T)

# Get full trajectory (robot + pendulum joints)
# Note: mpc_instance stores q_traj_full if we saved it
# For now, we'll reconstruct from joint_positions
print("\n⚠ Note: Full trajectory animation requires storing q_traj_full in run_mpc_goals()")
print("   The current implementation only stores robot joints.")
print("   To enable full pendulum animation, modify mpc_controller.py to store full state.")

# Display final robot configuration
if len(stats_to_viz['joint_positions']) > 0:
    final_q = stats_to_viz['joint_positions'][-1]
    # Pad with zeros for pendulum DOFs (not tracked)
    q_full = np.zeros(model_viz.nq)
    q_full[:len(final_q)] = final_q
    viz.display(q_full)
    print("\n✓ Displaying final robot configuration")

## Trajectory Analysis

Compare tracking performance across different batch sizes.

In [ ]:
# Compare end-effector trajectory components
fig, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

for batch_size, stats, color, label in [
    (1, mpc_stats_1, 'red', 'Batch 1'),
    (32, mpc_stats_32, 'green', 'Batch 32'),
    (128, mpc_stats_128, 'blue', 'Batch 128')
]:
    t = stats['timestamps']
    ee = stats['ee_actual']
    
    axs[0].plot(t, ee[:, 0], label=label, linewidth=2, color=color, alpha=0.8)
    axs[1].plot(t, ee[:, 1], label=label, linewidth=2, color=color, alpha=0.8)
    axs[2].plot(t, ee[:, 2], label=label, linewidth=2, color=color, alpha=0.8)

axs[0].set_ylabel('X [m]', fontsize=12)
axs[1].set_ylabel('Y [m]', fontsize=12)
axs[2].set_ylabel('Z [m]', fontsize=12)
axs[2].set_xlabel('Time [s]', fontsize=12)

for ax in axs:
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best')

fig.suptitle('End-Effector Trajectory Comparison', fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

In [ ]:
# Helper function to segment stats by goal
def segments_from_stats(stats):
    """Extract trajectory segments for each goal."""
    if 'ee_goal' not in stats or len(stats['ee_goal']) == 0:
        return []
    
    ee_goal = np.asarray(stats['ee_goal'])
    T = ee_goal.shape[0]
    
    # Find where goal changes
    change_idxs = [0]
    for i in range(1, T):
        if not np.allclose(ee_goal[i], ee_goal[i-1], atol=1e-9):
            change_idxs.append(i)
    change_idxs.append(T)
    
    segments = []
    for s, e in zip(change_idxs[:-1], change_idxs[1:]):
        segments.append({'start': s, 'end': e, 'goal': ee_goal[s]})
    
    return segments

# Get segments for each batch size
seg1 = segments_from_stats(mpc_stats_1)
seg32 = segments_from_stats(mpc_stats_32)
seg128 = segments_from_stats(mpc_stats_128)

print(f"Segments found: batch_1={len(seg1)}, batch_32={len(seg32)}, batch_128={len(seg128)}")

In [ ]:
# Plot 3D trajectory for each goal with projections
num_goals = max(len(seg1), len(seg32), len(seg128))

EE1 = np.asarray(mpc_stats_1['ee_actual'])
EE32 = np.asarray(mpc_stats_32['ee_actual'])
EE128 = np.asarray(mpc_stats_128['ee_actual'])

for goal_idx in range(num_goals):
    fig = plt.figure(figsize=(14, 9))
    
    # 3D trajectory
    ax_3d = fig.add_subplot(2, 2, 1, projection='3d')
    ax_xy = fig.add_subplot(2, 2, 2)
    ax_xz = fig.add_subplot(2, 2, 3)
    ax_yz = fig.add_subplot(2, 2, 4)
    
    goal_pt = None
    
    # Plot batch 1
    if goal_idx < len(seg1):
        s, e = seg1[goal_idx]['start'], seg1[goal_idx]['end']
        traj = EE1[s:e]
        ax_3d.plot(traj[:, 0], traj[:, 1], traj[:, 2], 'r-', linewidth=2, label='Batch 1', alpha=0.8)
        ax_xy.plot(traj[:, 0], traj[:, 1], 'r-', linewidth=2, label='Batch 1', alpha=0.8)
        ax_xz.plot(traj[:, 0], traj[:, 2], 'r-', linewidth=2, label='Batch 1', alpha=0.8)
        ax_yz.plot(traj[:, 1], traj[:, 2], 'r-', linewidth=2, label='Batch 1', alpha=0.8)
        goal_pt = seg1[goal_idx]['goal']
    
    # Plot batch 32
    if goal_idx < len(seg32):
        s, e = seg32[goal_idx]['start'], seg32[goal_idx]['end']
        traj = EE32[s:e]
        ax_3d.plot(traj[:, 0], traj[:, 1], traj[:, 2], 'g-', linewidth=2, label='Batch 32', alpha=0.8)
        ax_xy.plot(traj[:, 0], traj[:, 1], 'g-', linewidth=2, label='Batch 32', alpha=0.8)
        ax_xz.plot(traj[:, 0], traj[:, 2], 'g-', linewidth=2, label='Batch 32', alpha=0.8)
        ax_yz.plot(traj[:, 1], traj[:, 2], 'g-', linewidth=2, label='Batch 32', alpha=0.8)
        if goal_pt is None:
            goal_pt = seg32[goal_idx]['goal']
    
    # Plot batch 128
    if goal_idx < len(seg128):
        s, e = seg128[goal_idx]['start'], seg128[goal_idx]['end']
        traj = EE128[s:e]
        ax_3d.plot(traj[:, 0], traj[:, 1], traj[:, 2], 'b-', linewidth=2, label='Batch 128', alpha=0.8)
        ax_xy.plot(traj[:, 0], traj[:, 1], 'b-', linewidth=2, label='Batch 128', alpha=0.8)
        ax_xz.plot(traj[:, 0], traj[:, 2], 'b-', linewidth=2, label='Batch 128', alpha=0.8)
        ax_yz.plot(traj[:, 1], traj[:, 2], 'b-', linewidth=2, label='Batch 128', alpha=0.8)
        if goal_pt is None:
            goal_pt = seg128[goal_idx]['goal']
    
    # Mark goal
    if goal_pt is not None:
        ax_3d.scatter(*goal_pt, marker='*', s=200, c='k', label='Goal', zorder=10)
        ax_xy.scatter(goal_pt[0], goal_pt[1], marker='*', s=200, c='k', label='Goal', zorder=10)
        ax_xz.scatter(goal_pt[0], goal_pt[2], marker='*', s=200, c='k', label='Goal', zorder=10)
        ax_yz.scatter(goal_pt[1], goal_pt[2], marker='*', s=200, c='k', label='Goal', zorder=10)
    
    # Configure plots
    ax_3d.set_xlabel('X [m]')
    ax_3d.set_ylabel('Y [m]')
    ax_3d.set_zlabel('Z [m]')
    ax_3d.legend()
    ax_3d.set_title('3D Trajectory')
    
    for ax, title, xlabel, ylabel in [
        (ax_xy, 'X-Y Projection', 'X [m]', 'Y [m]'),
        (ax_xz, 'X-Z Projection', 'X [m]', 'Z [m]'),
        (ax_yz, 'Y-Z Projection', 'Y [m]', 'Z [m]')
    ]:
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.axis('equal')
    
    if goal_pt is not None:
        fig.suptitle(f'Goal {goal_idx}: [{goal_pt[0]:.2f}, {goal_pt[1]:.2f}, {goal_pt[2]:.2f}] m', 
                     fontsize=14)
    
    plt.tight_layout()
    plt.show()

## Performance Summary

In [ ]:
# Print summary table
print("\n" + "="*80)
print("PERFORMANCE SUMMARY")
print("="*80)
print(f"{'Batch':<10} {'Goals':<15} {'Time (s)':<15} {'Avg Solve (ms)':<20}")
print("-"*80)

for batch_size, stats in [
    (1, mpc_stats_1),
    (8, mpc_stats_8),
    (32, mpc_stats_32),
    (128, mpc_stats_128)
]:
    goals_reached = sum(1 for o in stats['goal_outcomes'] if o == 'reached')
    goals_str = f"{goals_reached}/{len(goals)}"
    
    time_all = stats.get('time_to_all_reached', np.nan)
    time_str = f"{time_all:.2f}" if not np.isnan(time_all) else "timeout"
    
    avg_solve = np.mean(stats['solve_times'])
    std_solve = np.std(stats['solve_times'])
    
    print(f"{batch_size:<10} {goals_str:<15} {time_str:<15} {avg_solve:.3f} ± {std_solve:.3f}")

print("\n" + "="*80)

In [ ]:
# Plot solve times over time
fig, ax = plt.subplots(figsize=(10, 5))

for batch_size, stats, color, label in [
    (1, mpc_stats_1, 'red', 'Batch 1'),
    (8, mpc_stats_8, 'orange', 'Batch 8'),
    (32, mpc_stats_32, 'green', 'Batch 32'),
    (128, mpc_stats_128, 'blue', 'Batch 128')
]:
    ax.plot(stats['timestamps'], stats['solve_times'], 
            label=label, alpha=0.7, linewidth=1.5, color=color)

ax.set_xlabel('Time [s]', fontsize=12)
ax.set_ylabel('Solve Time [ms]', fontsize=12)
ax.set_title('MPC Solve Time Over Episode', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Notes

**Observations**:
- Larger batch sizes provide better disturbance rejection through force estimation
- Trade-off between solve time and robustness
- Batch 1 has no force estimation (baseline)
- Batch 128 explores more force hypotheses

**Next Steps**:
- Run `benchmark_pickplace.py` for systematic evaluation
- Run `sweep_pickplace.py` for parameter sweep studies
- Analyze results with `case_study_3.ipynb` analysis code